<h1>Imports</h1>

In [1]:
# Importing necessary packages

import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

<h1>Supplementary Functions</h1>

In [2]:
# Calculating angle between joints

def calculate_angle(point1, point2, point3):
    point1 = np.array(point1)
    point3 = np.array(point3)
    point2 = np.array(point2)
    
    radians = np.arctan2(point3[1] - point2[1], point3[0] - point2[0]) - np.arctan2(point1[1] - point2[1], point1[0] - point2[0])
    angle = np.abs((radians * 180.0)/np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
        
    return angle

<h2>Initial Testing to differentiate state angles</h2>

In [7]:
cap = cv2.VideoCapture("sample_videos/abdullah_footage_1.mp4")

elbow_angles = []
shoulder_angles = []
hip_angles = []
knee_angles = []

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            break
        
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        results = pose.process(image)
        image.flags.writeable = True
        
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        try:
            landmarks = results.pose_landmarks.landmark
            
            # For Right side Points Visibility
            
            right_elbow_vis = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].visibility
            right_shoulder_vis = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].visibility
            right_hip_vis = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].visibility
            right_knee_vis = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].visibility
            right_wrist_vis = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].visibility
            right_ankle_vis = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].visibility
            
            
            # For Left side Points Visibility
            
            left_elbow_vis = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].visibility
            left_shoulder_vis = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].visibility
            left_hip_vis = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].visibility
            left_knee_vis = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].visibility
            left_wrist_vis = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].visibility
            left_ankle_vis = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].visibility
            
            left_vis_sum = (left_hip_vis + left_knee_vis + left_ankle_vis + left_elbow_vis + left_wrist_vis + left_shoulder_vis) 
            
            # Right Keypoints
            
            right_elbow = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y
            right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y
            right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y
            right_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y
            right_knee = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y
            right_ankle = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y
            
            right_vis_sum = (right_hip_vis + right_knee_vis + right_ankle_vis + right_elbow_vis + right_wrist_vis + right_shoulder_vis)
            
            # Left Keypoints
            
            left_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y
            left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y
            left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y
            left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y
            left_knee = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y
            left_ankle = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y
            
                
            
            # Elbow Angle Calculation
            
            if right_vis_sum >= left_vis_sum:
                elbow_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
                elbow_angles.append(elbow_angle.round(1))
            else:
                elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
                elbow_angles.append(elbow_angle.round(1))
                
            
                
                
            # Shoulder Angle Calculation
            
            if right_vis_sum >= left_vis_sum:
                shoulder_angle = calculate_angle(right_hip, right_shoulder, right_elbow)
                shoulder_angles.append(shoulder_angle.round(1))
            else:
                shoulder_angle = calculate_angle(left_hip, left_shoulder, left_elbow)
                shoulder_angles.append(shoulder_angle.round(1))
        
                
                
            # Hip Angle Calculation
            
            if right_vis_sum >= left_vis_sum:
                hip_angle = calculate_angle(right_shoulder, right_hip, right_knee)
                hip_angles.append(hip_angle.round(1))
            else:
                hip_angle = calculate_angle(left_shoulder, left_hip, left_knee)
                hip_angles.append(hip_angle.round(1))
                
            
            # Knee Angle Calculation
            
            
            if right_vis_sum >= left_vis_sum:
                knee_angle = calculate_angle(right_hip, right_knee, right_ankle)
                knee_angles.append(knee_angle.round(1))
            else:
                knee_angle = calculate_angle(left_hip, left_knee, left_ankle)
                knee_angles.append(knee_angle.round(1))
                        
            
            
        except:
            pass 
        
        
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        
        cv2.imshow('Raw Webcam Feed', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
        
    cap.release()
    cv2.destroyAllWindows()
    
        
# print min and max angles for each joint

print("Elbow Angle: ", min(elbow_angles), max(elbow_angles))
print("Shoulder Angle: ", min(shoulder_angles), max(shoulder_angles))
print("Hip Angle: ", min(hip_angles), max(hip_angles))
print("Knee Angle: ", min(knee_angles), max(knee_angles))

Elbow Angle:  61.3 173.4
Shoulder Angle:  0.2 61.4
Hip Angle:  149.2 169.4
Knee Angle:  171.3 180.0


In [3]:
state1_angle_threshold = [150.0, 180.0]
state2_angle_threshold = [100.0, 130.0]
state3_angle_threshold = [50.0, 80.0]

<p>Using elbow angles will be better to transition through states.</p>
<p>This is because hips and knees barely move whereas elbow angles can vary throughout the movement.</p>

<h2>Getting Angles from Trainer Footage</h2>

In [4]:
trainer_elbow_angle_state1 = []
trainer_shoulder_angle_state1 = []
trainer_hip_angle_state1 = []
trainer_knee_angle_state1 = []

trainer_elbow_angle_state2 = []
trainer_shoulder_angle_state2 = []
trainer_hip_angle_state2 = []
trainer_knee_angle_state2 = []

trainer_elbow_angle_state3 = []
trainer_shoulder_angle_state3 = []
trainer_hip_angle_state3 = []
trainer_knee_angle_state3 = []

In [5]:
cap = cv2.VideoCapture("./sample_videos/abdullah_footage.mp4")

state = 0

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            break
        
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        results = pose.process(image)
        image.flags.writeable = True
        
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        try:
            landmarks = results.pose_landmarks.landmark
            
            # For Right side Points Visibility
            
            right_elbow_vis = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].visibility
            right_shoulder_vis = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].visibility
            right_hip_vis = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].visibility
            right_knee_vis = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].visibility
            right_wrist_vis = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].visibility
            right_ankle_vis = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].visibility
            
            right_vis_sum = (right_hip_vis + right_knee_vis + right_ankle_vis + right_elbow_vis + right_wrist_vis + right_shoulder_vis)
            
            # For Left side Points Visibility
            
            left_elbow_vis = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].visibility
            left_shoulder_vis = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].visibility
            left_hip_vis = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].visibility
            left_knee_vis = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].visibility
            left_wrist_vis = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].visibility
            left_ankle_vis = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].visibility
            
            left_vis_sum = (left_hip_vis + left_knee_vis + left_ankle_vis + left_elbow_vis + left_wrist_vis + left_shoulder_vis) 
            
            # Right Keypoints
            
            right_elbow = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y
            right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y
            right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y
            right_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y
            right_knee = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y
            right_ankle = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y
            
            
            # Left Keypoints
            
            left_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y
            left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y
            left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y
            left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y
            left_knee = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y
            left_ankle = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y
            
                
            
            # Elbow Angle Calculation and state selection
            
            elbow_angle = 0
            
            if right_vis_sum >= left_vis_sum:
                elbow_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
            else:
                elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)  
            
            
            if elbow_angle >= state1_angle_threshold[0] and elbow_angle <= state1_angle_threshold[1]:
                state = 1
            elif elbow_angle >= state2_angle_threshold[0] and elbow_angle <= state2_angle_threshold[1]:
                state = 2
            elif elbow_angle >= state3_angle_threshold[0] and elbow_angle <= state3_angle_threshold[1]:
                state = 3
            else:
                state = 0
                    
            if state == 1:
                trainer_elbow_angle_state1.append(elbow_angle.round(1))
            elif state == 2:
                trainer_elbow_angle_state2.append(elbow_angle.round(1))
            elif state == 3:
                trainer_elbow_angle_state3.append(elbow_angle.round(1))
            
                
            # Shoulder Angle Calculation
            
            shoulder_angle = 0
            
            if right_vis_sum >= left_vis_sum:
                shoulder_angle = calculate_angle(right_hip, right_shoulder, right_elbow)
            else:
                shoulder_angle = calculate_angle(left_hip, left_shoulder, left_elbow)
                
            if state == 1:
                trainer_shoulder_angle_state1.append(shoulder_angle.round(1))
            elif state == 2:
                trainer_shoulder_angle_state2.append(shoulder_angle.round(1))
            elif state == 3:
                trainer_shoulder_angle_state3.append(shoulder_angle.round(1)) 
            
                
            # Hip Angle Calculation
            
            hip_angle = 0
            
            if right_vis_sum >= left_vis_sum:
                hip_angle = calculate_angle(right_shoulder, right_hip, right_knee)
            else:
                hip_angle = calculate_angle(left_shoulder, left_hip, left_knee)
                
            if state == 1:
                trainer_hip_angle_state1.append(hip_angle.round(1))
            elif state == 2:
                trainer_hip_angle_state2.append(hip_angle.round(1))
            elif state == 3:
                trainer_hip_angle_state3.append(hip_angle.round(1))
            
            # Knee Angle Calculation
            
            knee_angle = 0
            
            if right_vis_sum >= left_vis_sum:
                knee_angle = calculate_angle(right_hip, right_knee, right_ankle)
            else:
                knee_angle = calculate_angle(left_hip, left_knee, left_ankle)
                        
            if state == 1:
                trainer_knee_angle_state1.append(knee_angle.round(1))
            elif state == 2:
                trainer_knee_angle_state2.append(knee_angle.round(1))
            elif state == 3:
                trainer_knee_angle_state3.append(knee_angle.round(1))
            
            
        except:
            pass 
        
        
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        
        cv2.imshow('Raw Webcam Feed', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
        
    cap.release()
    cv2.destroyAllWindows()

In [6]:
# print all trainer angle arrays

print("State 1:-\n")
print("Trainer Elbow Angle State 1: ", trainer_elbow_angle_state1)
print("Trainer Shoulder Angle State 1: ", trainer_shoulder_angle_state1)
print("Trainer Hip Angle State 1: ", trainer_hip_angle_state1)
print("Trainer Knee Angle State 1: ", trainer_knee_angle_state1)

print("\nState 2:-\n")
print("Trainer Elbow Angle State 2: ", trainer_elbow_angle_state2)
print("Trainer Shoulder Angle State 2: ", trainer_shoulder_angle_state2)
print("Trainer Hip Angle State 2: ", trainer_hip_angle_state2)
print("Trainer Knee Angle State 2: ", trainer_knee_angle_state2)

print("\nState 3:-\n")
print("Trainer Elbow Angle State 3: ", trainer_elbow_angle_state3)
print("Trainer Shoulder Angle State 3: ", trainer_shoulder_angle_state3)
print("Trainer Hip Angle State 3: ", trainer_hip_angle_state3)
print("Trainer Knee Angle State 3: ", trainer_knee_angle_state3)


State 1:-

Trainer Elbow Angle State 1:  [163.4, 163.8, 164.6, 164.7, 164.8, 164.6, 164.2, 163.6, 163.0, 162.2, 160.4, 159.3, 156.4, 152.1, 156.4, 162.0, 168.7, 171.4, 170.5, 169.6, 169.2, 169.6, 169.9, 170.0, 170.1, 170.1, 170.1, 169.9, 167.6, 162.8, 157.2, 151.1, 154.0, 159.7, 166.0, 169.6, 170.3, 169.9, 171.0, 171.3, 171.2, 171.1, 171.1, 170.9, 170.7, 170.2, 168.1, 163.8, 158.8, 153.3, 155.8, 161.0, 167.0, 170.2, 170.6, 169.7, 169.9, 170.0, 169.8, 169.7, 169.6, 169.5, 168.2, 164.7, 159.4, 153.6, 154.2, 160.4, 166.4, 170.3, 171.1, 171.0]
Trainer Shoulder Angle State 1:  [53.3, 53.6, 54.6, 54.4, 54.7, 54.6, 54.4, 54.1, 53.8, 53.5, 52.9, 53.1, 53.6, 52.9, 40.2, 43.5, 45.4, 47.1, 48.4, 49.1, 50.0, 50.8, 50.9, 51.1, 51.7, 51.8, 51.7, 51.5, 50.6, 48.8, 47.1, 42.8, 40.5, 41.6, 45.5, 48.2, 50.7, 51.8, 52.7, 52.4, 52.1, 52.6, 52.2, 52.5, 52.1, 51.5, 50.3, 48.8, 48.1, 45.5, 42.1, 44.5, 46.6, 48.7, 50.5, 51.5, 51.6, 52.6, 52.7, 52.9, 53.0, 53.3, 52.6, 51.5, 50.7, 47.4, 42.7, 44.3, 46.5, 49.1, 

In [7]:
# print min and max angles for each joint

print("Min and max of angles:-\n")
print("\nState 1:-\n")
print("Elbow Angle: ", min(trainer_elbow_angle_state1), max(trainer_elbow_angle_state1))
print("Shoulder Angle: ", min(trainer_shoulder_angle_state1), max(trainer_shoulder_angle_state1))
print("Hip Angle: ", min(trainer_hip_angle_state1), max(trainer_hip_angle_state1))
print("Knee Angle: ", min(trainer_knee_angle_state1), max(trainer_knee_angle_state1))

print("\nState 2:-\n")
print("Elbow Angle: ", min(trainer_elbow_angle_state2), max(trainer_elbow_angle_state2))
print("Shoulder Angle: ", min(trainer_shoulder_angle_state2), max(trainer_shoulder_angle_state2))
print("Hip Angle: ", min(trainer_hip_angle_state2), max(trainer_hip_angle_state2))
print("Knee Angle: ", min(trainer_knee_angle_state2), max(trainer_knee_angle_state2))

print("\nState 3:-\n")
print("Elbow Angle: ", min(trainer_elbow_angle_state3), max(trainer_elbow_angle_state3))
print("Shoulder Angle: ", min(trainer_shoulder_angle_state3), max(trainer_shoulder_angle_state3))
print("Hip Angle: ", min(trainer_hip_angle_state3), max(trainer_hip_angle_state3))
print("Knee Angle: ", min(trainer_knee_angle_state3), max(trainer_knee_angle_state3))

# put min max in variables

elbow_angle_state1_min = min(trainer_elbow_angle_state1)
elbow_angle_state1_max = max(trainer_elbow_angle_state1)

shoulder_angle_state1_min = min(trainer_shoulder_angle_state1)
shoulder_angle_state1_max = max(trainer_shoulder_angle_state1)

hip_angle_state1_min = min(trainer_hip_angle_state1)
hip_angle_state1_max = max(trainer_hip_angle_state1)

knee_angle_state1_min = min(trainer_knee_angle_state1)
knee_angle_state1_max = max(trainer_knee_angle_state1)

elbow_angle_state2_min = min(trainer_elbow_angle_state2)
elbow_angle_state2_max = max(trainer_elbow_angle_state2)

shoulder_angle_state2_min = min(trainer_shoulder_angle_state2)
shoulder_angle_state2_max = max(trainer_shoulder_angle_state2)

hip_angle_state2_min = min(trainer_hip_angle_state2)
hip_angle_state2_max = max(trainer_hip_angle_state2)

knee_angle_state2_min = min(trainer_knee_angle_state2)
knee_angle_state2_max = max(trainer_knee_angle_state2)

elbow_angle_state3_min = min(trainer_elbow_angle_state3)
elbow_angle_state3_max = max(trainer_elbow_angle_state3)

shoulder_angle_state3_min = min(trainer_shoulder_angle_state3)
shoulder_angle_state3_max = max(trainer_shoulder_angle_state3)

hip_angle_state3_min = min(trainer_hip_angle_state3)
hip_angle_state3_max = max(trainer_hip_angle_state3)

knee_angle_state3_min = min(trainer_knee_angle_state3)
knee_angle_state3_max = max(trainer_knee_angle_state3)

Min and max of angles:-


State 1:-

Elbow Angle:  151.1 171.4
Shoulder Angle:  40.2 54.7
Hip Angle:  163.0 179.6
Knee Angle:  171.8 180.0

State 2:-

Elbow Angle:  100.8 129.9
Shoulder Angle:  10.3 40.1
Hip Angle:  166.3 179.8
Knee Angle:  171.4 178.1

State 3:-

Elbow Angle:  50.0 79.0
Shoulder Angle:  1.6 28.4
Hip Angle:  165.6 176.8
Knee Angle:  172.9 179.9


<h1>Comparing trainer skeleton to user and evaluating</h1>

In [63]:
# Variables

leniency = 23
errors = []
error_bool = False
depth_bool = False
elbow_angle = 0
shoulder_angle = 0
hip_angle = 0
knee_angle = 0
reps = 0


# Test Variables

client_incorrect = []
trainer_incorrect = []

In [64]:
cap = cv2.VideoCapture('./sample_videos/leena_footage.mp4')

state = 0
states_visited = []

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            break
        
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        results = pose.process(image)
        image.flags.writeable = True
        
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        try:
            landmarks = results.pose_landmarks.landmark
            
            error_bool = False
            
            
            # For Right side Points Visibility
            
            right_elbow_vis = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].visibility
            right_shoulder_vis = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].visibility
            right_hip_vis = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].visibility
            right_knee_vis = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].visibility
            right_wrist_vis = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].visibility
            right_ankle_vis = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].visibility
            
            right_vis_sum = (right_hip_vis + right_knee_vis + right_ankle_vis + right_elbow_vis + right_wrist_vis + right_shoulder_vis)
            
            # For Left side Points Visibility
            
            left_elbow_vis = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].visibility
            left_shoulder_vis = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].visibility
            left_hip_vis = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].visibility
            left_knee_vis = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].visibility
            left_wrist_vis = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].visibility
            left_ankle_vis = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].visibility
            
            left_vis_sum = (left_hip_vis + left_knee_vis + left_ankle_vis + left_elbow_vis + left_wrist_vis + left_shoulder_vis) 
            
            
            # Right Keypoints
            
            right_elbow = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y
            right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y
            right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y
            right_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y
            right_knee = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y
            right_ankle = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y
            
            # Left Keypoints
            
            left_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y
            left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y
            left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y
            left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y
            left_knee = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y
            left_ankle = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y
            
            
            
            # Elbow Angle Calculation
            
            
            if right_vis_sum >= left_vis_sum:
                elbow_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
            else:
                elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
             
           
           # State Selection
           
            if elbow_angle >= state1_angle_threshold[0] and elbow_angle <= state1_angle_threshold[1]:
                state = 1
                states_visited.append(1)
                
                if len(states_visited) == 0:
                    states_visited.append(1)
                elif states_visited.__contains__(1) and states_visited.__contains__(2) and states_visited.__contains__(3):
                    states_visited = []
                    states_visited.append(1)
                    depth_bool = False
                    reps += 1
                    print("Reps: ", reps)
                elif states_visited.__contains__(1) and states_visited.__contains__(2):
                    states_visited = []
                    errors.append("State 3 not visited, hit more depth")
                    error_bool = True
                    depth_bool = True
                    state = 0
                    client_incorrect.append("Full range of motion not performed")
                    trainer_incorrect.append("Full range of motion not performed")
                    reps += 1
                    print("Reps: ", reps)
                    
                
            elif elbow_angle >= state2_angle_threshold[0] and elbow_angle <= state2_angle_threshold[1]:
                state = 2
                states_visited.append(2)
                
            elif elbow_angle >= state3_angle_threshold[0] and elbow_angle <= state3_angle_threshold[1]:
                state = 3
                states_visited.append(3)
            else:
                state = 0
                states_visited.append(0)
            
                    
            # Shoulder Angle Calculation
            
            if right_vis_sum >= left_vis_sum:
                shoulder_angle = calculate_angle(right_hip, right_shoulder, right_elbow)
            else:
                shoulder_angle = calculate_angle(left_hip, left_shoulder, left_elbow)
        
                
                
            # Hip Angle Calculation
            
            if right_vis_sum >= left_vis_sum:
                hip_angle = calculate_angle(right_shoulder, right_hip, right_knee)
            else:
                hip_angle = calculate_angle(left_shoulder, left_hip, left_knee)
                
            
            
            # Knee Angle Calculation
            
            
            if right_vis_sum >= left_vis_sum:
                knee_angle = calculate_angle(right_hip, right_knee, right_ankle)
            else:
                knee_angle = calculate_angle(left_hip, left_knee, left_ankle)
            
            
            # Checking Angles according to state
            
            trainer_elbow_min = 0
            trainer_elbow_max = 0
            
            trainer_shoulder_min = 0
            trainer_shoulder_max = 0
            
            trainer_hip_min = 0
            trainer_hip_max = 0
            
            trainer_knee_min = 0
            trainer_knee_max = 0
            
            if state == 1:
                trainer_elbow_min = elbow_angle_state1_min
                trainer_elbow_max = elbow_angle_state1_max
                
                trainer_shoulder_min = shoulder_angle_state1_min
                trainer_shoulder_max = shoulder_angle_state1_max
                
                trainer_hip_min = hip_angle_state1_min
                trainer_hip_max = hip_angle_state1_max
                
                trainer_knee_min = knee_angle_state1_min
                trainer_knee_max = knee_angle_state1_max

                
            elif state == 2:
                trainer_elbow_min = elbow_angle_state2_min
                trainer_elbow_max = elbow_angle_state2_max
                
                trainer_shoulder_min = shoulder_angle_state2_min
                trainer_shoulder_max = shoulder_angle_state2_max
                
                trainer_hip_min = hip_angle_state2_min
                trainer_hip_max = hip_angle_state2_max
                
                trainer_knee_min = knee_angle_state2_min
                trainer_knee_max = knee_angle_state2_max
                
            elif state == 3:
                trainer_elbow_min = elbow_angle_state3_min
                trainer_elbow_max = elbow_angle_state3_max
                
                trainer_shoulder_min = shoulder_angle_state3_min
                trainer_shoulder_max = shoulder_angle_state3_max
                
                trainer_hip_min = hip_angle_state3_min
                trainer_hip_max = hip_angle_state3_max
                
                trainer_knee_min = knee_angle_state3_min
                trainer_knee_max = knee_angle_state3_max
                
            else:
                trainer_elbow_min = 0
                trainer_elbow_max = 0
                
                trainer_shoulder_min = 0
                trainer_shoulder_max = 0
                
                trainer_hip_min = 0
                trainer_hip_max = 0
                
                trainer_knee_min = 0
                trainer_knee_max = 0
            
            
            # Perform checking of errors when user is not in state 0
            
            if state != 0: 
                
                # Elbow angle matching
            
                if elbow_angle < (trainer_elbow_min - leniency):
                    errors.append("Elbow Angle too low")
                    error_bool = True
                    client_incorrect.append(elbow_angle.round(1))
                    trainer_incorrect.append(trainer_elbow_min - leniency)
                elif elbow_angle > (trainer_elbow_max + leniency):
                    errors.append("Elbow Angle too high")
                    error_bool = True
                    client_incorrect.append(elbow_angle.round(1))
                    trainer_incorrect.append(trainer_elbow_max + leniency)
                
                
                
                # Shoulder angle matching
                
                if shoulder_angle < (trainer_shoulder_min - leniency):
                    errors.append("Shoulder Angle too low")
                    error_bool = True
                    client_incorrect.append(shoulder_angle.round(1))
                    trainer_incorrect.append(trainer_shoulder_min - leniency)
                elif shoulder_angle > (trainer_shoulder_max + leniency):
                    errors.append("Shoulder Angle too high")
                    error_bool = True
                    client_incorrect.append(shoulder_angle.round(1))
                    trainer_incorrect.append(trainer_shoulder_max + leniency)
                
                
                
                # Hip Angle Matching
                
                if hip_angle < (trainer_hip_min - leniency):
                    errors.append("Hip Angle too low")
                    error_bool = True
                    client_incorrect.append(hip_angle.round(1))
                    trainer_incorrect.append(trainer_hip_min - leniency)
                elif hip_angle > (trainer_hip_max + leniency):
                    errors.append("Hip Angle too high")
                    error_bool = True
                    client_incorrect.append(hip_angle.round(1))
                    trainer_incorrect.append(trainer_hip_max + leniency)


                # Knee Angle Matching
                
                if knee_angle < (trainer_knee_min - leniency):
                    errors.append("Knee Angle too low")
                    error_bool = True
                    client_incorrect.append(knee_angle.round(1))
                    trainer_incorrect.append(trainer_knee_min - leniency)
                elif knee_angle > (trainer_knee_max + leniency):
                    errors.append("Knee Angle too high")
                    error_bool = True
                    client_incorrect.append(knee_angle.round(1))
                    trainer_incorrect.append(trainer_knee_max + leniency)
                
            
        except:
            pass 
        
        if depth_bool == True:
            cv2.putText(image, "Increase Depth", (600, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS
                                        , mp_drawing.DrawingSpec(color=(0, 0, 0), thickness=2, circle_radius=2)
                                        , mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2)
                                      )
        else:
            cv2.putText(image, "Depth Perfect", (600, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS
                                        , mp_drawing.DrawingSpec(color=(0, 0, 0), thickness=2, circle_radius=2)
                                        , mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2)
                                      )
            
                    
        if error_bool == True:
            cv2.putText(image, "Error", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS
                                        , mp_drawing.DrawingSpec(color=(0, 0, 0), thickness=2, circle_radius=2)
                                        , mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2)
                                      )
            
        else:
            cv2.putText(image, "No Error", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS
                                        , mp_drawing.DrawingSpec(color=(0, 0, 0), thickness=2, circle_radius=2)
                                        , mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2)
                                      )
            
        
        
        cv2.imshow('Raw Webcam Feed', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
        
    cap.release()
    cv2.destroyAllWindows()

Reps:  1
Reps:  2
Reps:  3
Reps:  4


In [65]:
for i in range(len(errors)):
    print(errors[i] + " => Client: " + str(client_incorrect[i]) + " vs Trainer: " + str(trainer_incorrect[i]))

State 3 not visited, hit more depth => Client: Full range of motion not performed vs Trainer: Full range of motion not performed
Hip Angle too low => Client: 142.1 vs Trainer: 143.3
Hip Angle too low => Client: 139.9 vs Trainer: 143.3
Hip Angle too low => Client: 136.8 vs Trainer: 143.3
Hip Angle too low => Client: 137.3 vs Trainer: 143.3
Hip Angle too low => Client: 135.9 vs Trainer: 143.3
State 3 not visited, hit more depth => Client: Full range of motion not performed vs Trainer: Full range of motion not performed
Knee Angle too low => Client: 148.6 vs Trainer: 148.8
Knee Angle too low => Client: 148.5 vs Trainer: 148.8
Knee Angle too low => Client: 148.4 vs Trainer: 148.8
Knee Angle too low => Client: 148.6 vs Trainer: 148.8
Knee Angle too low => Client: 148.1 vs Trainer: 148.8
Knee Angle too low => Client: 147.9 vs Trainer: 148.8
Knee Angle too low => Client: 147.4 vs Trainer: 148.8
Knee Angle too low => Client: 147.9 vs Trainer: 148.8
Knee Angle too low => Client: 148.5 vs Traine